In [4]:
from langchain.llms.bedrock import Bedrock
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import BedrockEmbeddings
import boto3
import botocore
# os.environ['LANGCHAIN_TRACING_V2']='true'
# os.environ['LANGCHAIN_ENDPOINT']="https://api.smith.langchain.com"
# os.environ['LANGCHAIN_API_KEY']="ls__cb0ee22e50e34241896c1c8230a9e218"
# os.environ['LANGCHAIN_PROJECT']="demo"

config = botocore.config.Config(
    read_timeout=900,
    connect_timeout=900,
    retries={"max_attempts": 0}
)

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=config,
)
file = (r"C:\Users\Lenovo\Documents\Project-vs code\Amazon Transcribe\2-Cloumn page\WET_2013041616211199.pdf")
loader = PyPDFLoader(file)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=600,chunk_overlap=0)
texts = text_splitter.split_documents(docs)

embeddings = BedrockEmbeddings()
db = FAISS.from_documents(documents=texts,embedding=embeddings)

template = """
Answer truthfull based on the given text only


{context}
{question}
"""

retriever = db.as_retriever(search_type='similarity', search_kwargs={"k": 3})
llm = Bedrock(model_id="anthropic.claude-v2:1",client=bedrock_client,model_kwargs = {"temperature":1e-10,"max_tokens_to_sample": 8191})
qa_prompt = PromptTemplate(template=template, input_variables=["context","question"])
chain_type_kwargs = { "prompt": qa_prompt, "verbose": False }
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs=chain_type_kwargs,
    verbose=False
)